[Reference](https://towardsdatascience.com/using-causal-ml-instead-of-a-b-testing-eeb1067d7fc0)

In [2]:
import pandas as pd
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor

# initialize the number of folds and a dictionary to store the folds
n_folds = 5
folds = {fold: dict() for fold in range(n_folds)}

# for each fold...
for fold, (ix_train, ix_test) in enumerate(KFold(n_splits=n_folds).split(X=X)):

  # ... store test index and trained model
  folds[fold]["ix_test"] = ix_test
  folds[fold]["model"] = LGBMRegressor().fit(
    X=X.loc[ix_train, :], 
    y=y.loc[ix_train]
  )

In [4]:
# make counterfactual datasets for campaign A
X_zeros = X.replace({"campaign_A": {1: 0}}) # universe A: nobody gets the discount
X_ones = X.replace({"campaign_A": {0: 1}}) # universe B: everybody gets the discount

pred_zeros = pd.Series(index = X.index)
pred_ones = pd.Series(index = X.index)

# for each fold, use model to make predictions on test individuals and store them
for fold in folds.keys():
  
  ix_test = folds[fold]["ix_test"]
  model = folds[fold]["model"]
  
  pred_zeros.loc[ix_test] = model.predict(X_zeros.loc[ix_test, :])
  pred_ones.loc[ix_test] = model.predict(X_ones.loc[ix_test, :])

In [5]:
ate = (pred_ones - pred_zeros).mean()
ate